In [1]:
import inspect
from berny import Berny, geomlib, coords,Geometry,Math
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.berny_solver import to_berny_geom as tbg
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=3)

In [2]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2])),decimals=3)

In [3]:
h2o=gto.M(atom="O 0 0 0; H 0 -1.3 .4; H 0 1.3 .4",unit="Bohr",basis="def2-TZVP")

In [4]:
(1.3**2+.16)**.5

1.3601470508735443

In [5]:
mf=scf.RHF(h2o)
E0=mf.scf()
g0=mf.Gradients().grad()
h0=mf.Hessian().hess()

converged SCF energy = -75.8573023860845
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0000000000    -0.0000000000     0.3969639449
1 H     0.0000000000     0.5354215144    -0.1984819725
2 H     0.0000000000    -0.5354215144    -0.1984819725
----------------------------------------------


In [6]:
h0=h0.swapaxes(1,2)

In [7]:
h0=h0.reshape(9,9)

In [8]:
h2occords=h2o.atom_coords()

In [9]:
geom2=tbg(h2o)

In [10]:
anglecenter(h2occords),norm(h2occords[1])

(2.545, 1.3601470508735443)

In [11]:
b2=Berny(geom2)

In [12]:
s=b2._state
vars(s)

{'geom': <Geometry 'H2O'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 2, angles: 1, dihedrals: 0">,
 'H': array([[0.64 , 0.   , 0.   ],
        [0.   , 0.64 , 0.   ],
        [0.   , 0.   , 0.303]]),
 'weights': array([1.421, 1.421, 0.874]),
 'future': Point(q=array([1.36 , 1.36 , 2.545]), E=None, g=None),
 'first': True}

In [13]:
s.coords._coords,vars(s.coords._coords[0])

([Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)],
 {'i': 0, 'j': 1, 'idx': (0, 1), 'weak': 0})

In [14]:
s.H  #use to get second derivs 

array([[0.64 , 0.   , 0.   ],
       [0.   , 0.64 , 0.   ],
       [0.   , 0.   , 0.303]])

In [15]:
vars(geom2)   # coords are in angstrom

{'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
        [ 0.   , -0.688,  0.212],
        [ 0.   ,  0.688,  0.212]]), 'lattice': None}

In [16]:
geom2.coords*ang2bohr

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.3,  0.4],
       [ 0. ,  1.3,  0.4]])

In [17]:
b2._state.future.q  # bohr and radians

array([1.36 , 1.36 , 2.545])

In [18]:
# get derivatives angles independently
# 

In [19]:
s.future.q

array([1.36 , 1.36 , 2.545])

In [20]:
print(s.coords.B_matrix(s.geom))

[[ 0.     0.956 -0.294  0.    -0.956  0.294  0.     0.     0.   ]
 [ 0.    -0.956 -0.294  0.     0.     0.     0.     0.956  0.294]
 [ 0.     0.     1.405  0.    -0.216 -0.703  0.     0.216 -0.703]]


In [21]:
gsc=b2._state.geom.supercell()

In [22]:
vars(gsc),vars(s.geom) 

({'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
         [ 0.   , -0.688,  0.212],
         [ 0.   ,  0.688,  0.212]]), 'lattice': None},
 {'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
         [ 0.   , -0.688,  0.212],
         [ 0.   ,  0.688,  0.212]]), 'lattice': None})

In [23]:
[print(x) for x in enumerate(s.coords)]

(0, Bond(0, 1, weak=0))
(1, Bond(0, 2, weak=0))
(2, Angle(1, 0, 2, weak=0))


[None, None, None]

In [24]:
B=s.coords.B_matrix(s.geom)

In [25]:
for i,x in enumerate(s.coords):
    print(x.eval(gsc.coords, grad=True)[1],"\n \n")

[array([ 0.   ,  0.956, -0.294]), array([-0.   , -0.956,  0.294])] 
 

[array([ 0.   , -0.956, -0.294]), array([-0.   ,  0.956,  0.294])] 
 

[array([-0.   , -0.216, -0.703]), array([0.   , 0.   , 1.405]), array([-0.   ,  0.216, -0.703])] 
 



In [26]:
B

array([[ 0.   ,  0.956, -0.294,  0.   , -0.956,  0.294,  0.   ,  0.   ,
         0.   ],
       [ 0.   , -0.956, -0.294,  0.   ,  0.   ,  0.   ,  0.   ,  0.956,
         0.294],
       [ 0.   ,  0.   ,  1.405,  0.   , -0.216, -0.703,  0.   ,  0.216,
        -0.703]])

In [27]:
s.coords.eval_geom(s.geom)

array([1.36 , 1.36 , 2.545])

In [28]:
s.coords

<InternalCoords "bonds: 2, angles: 1, dihedrals: 0">

In [29]:
vars(coords.InternalCoords(s.geom, dihedral=s.params['dihedral'], superweakdih=s.params['superweakdih']))

{'_coords': [Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)],
 'fragments': [[0, 1, 2]]}

In [30]:
vars(coords.InternalCoords(s.geom, dihedral=s.params['dihedral'], superweakdih=s.params['superweakdih'])._coords[0])

{'i': 0, 'j': 1, 'idx': (0, 1), 'weak': 0}

In [31]:
vars(s.geom)

{'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
        [ 0.   , -0.688,  0.212],
        [ 0.   ,  0.688,  0.212]]), 'lattice': None}

In [32]:
B.T

array([[ 0.   ,  0.   ,  0.   ],
       [ 0.956, -0.956,  0.   ],
       [-0.294, -0.294,  1.405],
       [ 0.   ,  0.   ,  0.   ],
       [-0.956,  0.   , -0.216],
       [ 0.294,  0.   , -0.703],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.956,  0.216],
       [ 0.   ,  0.294, -0.703]])

In [33]:
s.future.q
(B.T@s.future.q).reshape(3,3)#-(B.T@s.future.q).reshape(3,3)[0]

array([[ 0.   ,  0.   ,  2.776],
       [ 0.   , -1.85 , -1.388],
       [ 0.   ,  1.85 , -1.388]])

In [34]:
s.geom.coords*ang2bohr

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.3,  0.4],
       [ 0. ,  1.3,  0.4]])

### praticamente qua mi d una proiezione dei modi normali sulle coordinate cartesiane della molecola, tutti insieme formano la B_matrix

In [35]:
a=np.array([-0.        , -0.91914503,  0.3939193 ])

In [36]:
norm(a)

1.0000000005430953

In [37]:
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))

In [38]:
B_inv,B_inv.shape,B.shape

(array([[ 0.   ,  0.   ,  0.   ],
        [ 0.338, -0.338,  0.   ],
        [-0.098, -0.098,  0.433],
        [ 0.   ,  0.   ,  0.   ],
        [-0.647, -0.309, -0.2  ],
        [ 0.101, -0.003, -0.217],
        [ 0.   ,  0.   ,  0.   ],
        [ 0.309,  0.647,  0.2  ],
        [-0.003,  0.101, -0.217]]), (9, 3), (3, 9))

In [39]:
np.dot(B_inv.T, g0.reshape(-1))

array([-0.57 , -0.57 ,  0.044])

In [109]:
s.H

array([[0.64 , 0.   , 0.   ],
       [0.   , 0.64 , 0.   ],
       [0.   , 0.   , 0.303]])

I can have the projection of the gradient in the internal coordinates as well as the approximated hessian matrix.

In [41]:
proj = np.dot(B, B_inv)
proj

array([[ 1.000e+00, -1.665e-16, -4.163e-17],
       [ 4.293e-17,  1.000e+00, -4.163e-17],
       [-2.255e-16, -3.331e-16,  1.000e+00]])

In [42]:
H_proj = proj.dot(s.H).dot(proj) + 1000 * (np.eye(len(s.coords)) - proj)
1000 * (np.eye(len(s.coords)) - proj)  #perhaps to avoid zeros in matrix inversion

array([[ 4.441e-13,  1.665e-13,  4.163e-14],
       [-4.293e-14,  0.000e+00,  4.163e-14],
       [ 2.255e-13,  3.331e-13, -2.220e-13]])

In [43]:
H_proj,s.H #differiscono per una manciata di infinesimi

(array([[ 6.396e-01,  1.663e-13,  4.159e-14],
        [-4.288e-14,  6.396e-01,  4.159e-14],
        [ 2.253e-13,  3.328e-13,  3.030e-01]]), array([[0.64 , 0.   , 0.   ],
        [0.   , 0.64 , 0.   ],
        [0.   , 0.   , 0.303]]))

In [218]:
def quadratic_step(g, H, w, trust):
    ev = np.linalg.eigvalsh((H + H.T) / 2)
    rfo = np.vstack((np.hstack((H, g[:, None])), np.hstack((g, 0))[None, :]))
    D, V = np.linalg.eigh((rfo + rfo.T) / 2)
    dq = V[:-1, 0] / V[-1, 0]
    l = D[0]
    print(dq,l,ev)
    if norm(dq) <= trust:
        print('Pure RFO step was performed:')
        on_sphere = False
    else:
        def steplength(l):
            return norm(np.linalg.solve(l * np.eye(H.shape[0]) - H, g)) - trust
        l = Math.findroot(steplength, ev[0])  # minimization on sphere
        dq = np.linalg.solve(l * np.eye(H.shape[0]) - H, g)
        on_sphere = True
        print('Minimization on sphere was performed:')
    dE = np.dot(g, dq) + 0.5 * dq.dot(H).dot(dq)
    return dq, dE, on_sphere

In [219]:
np.hstack((g0.flatten(), 0))

array([-3.519e-16, -3.553e-15,  3.970e-01,  5.648e-17,  5.354e-01,
       -1.985e-01,  2.955e-16, -5.354e-01, -1.985e-01,  0.000e+00])

In [220]:
current = Berny.Point(s.future.q, mf.e_tot, np.dot(B_inv.T, np.asarray(g0*bohr2ang).reshape(-1)))

In [221]:
current.g # also np.dot(B_inv.T, np.asarray(g0).reshape(-1)) is g in internal coords

array([-0.302, -0.302,  0.023])

In [222]:
 dq, dE, on_sphere = quadratic_step(np.dot(proj, current.g), H_proj, s.weights, s.trust)

[ 0.353  0.353 -0.045] -0.21424217441427845 [0.303 0.64  0.64 ]
Minimization on sphere was performed:


In [214]:
-np.linalg.inv(H_proj)@(np.dot(proj, current.g))

array([ 0.472,  0.472, -0.077])

In [215]:
np.linalg.solve(H_proj,np.dot(proj, current.g))

array([-0.472, -0.472,  0.077])

In [141]:
dq # change dq to the one from Morse potential fitting 

array([ 0.096,  0.096, -0.268])

In [49]:
current.q[0]

1.3601470508735443

## from dq in internal coords to dq in cartesian coords

In [50]:
q, a = s.coords.update_geom(s.geom, current.q, dq, B_inv)

In [51]:
q

array([1.572, 1.572, 2.523])

In [52]:
a.coords

array([[ 0.000e+00, -1.729e-14, -2.759e-02],
       [ 0.000e+00, -7.923e-01,  2.255e-01],
       [ 0.000e+00,  7.923e-01,  2.255e-01]])

In [53]:
(B.T@q).reshape(3,3)

array([[ 0.000e+00, -9.193e-14,  2.622e+00],
       [ 0.000e+00, -2.048e+00, -1.311e+00],
       [ 0.000e+00,  2.048e+00, -1.311e+00]])

In [54]:
b=a.coords
norm(b[1]-b[0]) #is angstrom

0.8317307886818113

In [55]:
B = s.coords.B_matrix(s.geom)
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
g_ic=np.dot(B_inv.T, (g0).reshape(-1))

In [56]:
g_ic

array([-0.57 , -0.57 ,  0.044])

In [57]:
B@h2occords.flatten()   # problems with angles

array([ 1.360e+00,  1.360e+00, -9.992e-16])

In [58]:
s.future

Point(q=array([1.36 , 1.36 , 2.545]), E=None, g=None)

In [59]:
B

array([[ 0.   ,  0.956, -0.294,  0.   , -0.956,  0.294,  0.   ,  0.   ,
         0.   ],
       [ 0.   , -0.956, -0.294,  0.   ,  0.   ,  0.   ,  0.   ,  0.956,
         0.294],
       [ 0.   ,  0.   ,  1.405,  0.   , -0.216, -0.703,  0.   ,  0.216,
        -0.703]])

In [60]:
h2occords

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.3,  0.4],
       [ 0. ,  1.3,  0.4]])

In [61]:
-0.956/ 0.294*.4,0.956**2+ 0.294**2

(-1.3006802721088437, 1.000372)

$$B_{\mu i}=\frac{\partial q_\mu}{\partial x_i}$$

In [62]:
g_ic=np.dot(B_inv.T, (g0).reshape(-1))
g_ic

array([-0.57 , -0.57 ,  0.044])

In [177]:
h_ic=B_inv.T@ h0@B_inv

In [114]:
h_ic

array([[ 2.705, -0.015,  0.007],
       [-0.015,  2.705,  0.007],
       [ 0.007,  0.007, -0.386]])

In [117]:
np.linalg.inv(h_ic)*ang2bohr

array([[ 6.987e-01,  3.966e-03,  1.206e-02],
       [ 3.966e-03,  6.987e-01,  1.206e-02],
       [ 1.206e-02,  1.206e-02, -4.896e+00]])

In [85]:
dq_pr=-np.linalg.inv(h_ic)@g_ic

In [120]:
np.set_printoptions(precision=3)

In [121]:
dq,dq_pr

(array([ 0.212,  0.212, -0.021]), array([0.212, 0.212, 0.121]))

In [182]:
s.future.q+quadratic_step(g_ic, h_ic+np.eye(3)*1e-6, s.weights, s.trust)[0]

Minimization on sphere was performed:


array([1.531, 1.531, 2.368])

In [183]:
pred_q=(s.future.q-np.linalg.inv(h_ic)@g_ic)
pred_q

array([1.572, 1.572, 2.665])

In [123]:
coords_new = h2occords+B_inv.dot(dq_pr).reshape(-1, 3)
coords_new-=coords_new[0]

In [124]:
norm(coords_new[2]),anglecenter(coords_new)

(1.5739906405950779, 2.649)

In [125]:
norm(coords_new[2]),anglecenter(coords_new)

(1.5739906405950779, 2.649)

In [126]:
h2occords-(np.linalg.inv(h0)@g0.flatten()).reshape(3,3)

array([[ 2.311e-12,  4.624e-02,  4.044e-01],
       [ 2.310e-12, -1.480e+00,  7.881e-01],
       [ 2.311e-12,  1.573e+00,  7.881e-01]])

In [127]:
h2occords

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.3,  0.4],
       [ 0. ,  1.3,  0.4]])

In [102]:
dxcc=-(np.linalg.inv(h0)@g0.flatten()).reshape(3,3)

In [103]:
dxcc-=dxcc[0]

In [130]:
anglecenter(h2occords)

2.545

In [165]:
1.491*180/np.pi

85.42800725400575